# F1 Pit Stop Feature Engineering

**Goal:** Transform raw pit stop data into ML-ready features

**Input:** `data/raw/f1_2023_pit_stops.csv` (930 pit stops)

**Output:** `data/processed/ml_features.csv` (with engineered features)

**Features we'll create:**
1. **Tire age** - How many laps on current compound
2. **Stop number** - Is this 1st, 2nd, or 3rd pit stop?
3. **Race progress** - What % of race is complete?
4. **Stint length** - How long between pit stops
5. **Position changes** - Gaining or losing places?


In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [2]:
# Load COMPLETE multi-season data
df = pd.read_csv('data/raw/f1_complete_2020_2024.csv')

print(f"📊 Complete dataset loaded!")
print(f"   Rows: {len(df)}")
print(f"   Columns: {len(df.columns)}")

# Show ALL column names
print(f"\n📋 ALL {len(df.columns)} columns:")
for i, col in enumerate(df.columns, 1):
    print(f"   {i:2}. {col}")

# Show year distribution
if 'Year' in df.columns:
    print(f"\n📅 Seasons included:")
    year_counts = df['Year'].value_counts().sort_index()
    for year, count in year_counts.items():
        print(f"      {year}: {count:4,} pit stops")
else:
    print("\n⚠️ Warning: 'Year' column not found!")

# Check for race-related columns
print(f"\n🔍 Race-related columns:")
race_cols = [col for col in df.columns if 'race' in col.lower() or 'round' in col.lower()]
if race_cols:
    for col in race_cols:
        print(f"   ✅ {col}")
else:
    print("   ⚠️ No race-related columns found!")

print(f"\n🔍 First few rows:")
print(df.head())

📊 Complete dataset loaded!
   Rows: 2628
   Columns: 33

📋 ALL 33 columns:
    1. Sector2Time
    2. FastF1Generated
    3. Sector2SessionTime
    4. LapStartTime
    5. Time
    6. TrackStatus
    7. Team
    8. DeletedReason
    9. Sector1SessionTime
   10. SpeedST
   11. LapStartDate
   12. RaceName
   13. PitInTime
   14. SpeedFL
   15. IsPersonalBest
   16. PitOutTime
   17. FreshTyre
   18. Position
   19. DriverNumber
   20. Deleted
   21. Year
   22. SpeedI2
   23. Compound
   24. Stint
   25. Driver
   26. TyreLife
   27. IsAccurate
   28. LapTime
   29. LapNumber
   30. Sector3Time
   31. Sector1Time
   32. SpeedI1
   33. Sector3SessionTime

📅 Seasons included:
      2020:  600 pit stops
      2021:  825 pit stops
      2023:  930 pit stops
      2024:  273 pit stops

🔍 Race-related columns:
   ✅ RaceName

🔍 First few rows:
              Sector2Time  FastF1Generated      Sector2SessionTime  \
0  0 days 00:00:42.629000            False  0 days 01:27:10.094000   
1  0 days 00:0

In [3]:
key_columns = ['Driver', 'LapNumber', 'LapTime', 'TyreLife', 
               'Compound', 'FreshTyre', 'Position', 'Stint', 
               'TrackStatus', 'RaceName', 'SpeedFL', 'Year']

df_features = df[key_columns].copy()

print("🔍 Key columns for ML (with TrackStatus!):")
print(df_features.head(10))

print(f"\n📊 Data types:")
print(df_features.dtypes)

print(f"\n❓ Missing values:")
print(df_features.isnull().sum())

print(f"\n🔍 FreshTyre distribution:")
print(df_features['FreshTyre'].value_counts())

print(f"\n🚦 TrackStatus distribution:")
print(df_features['TrackStatus'].value_counts())

🔍 Key columns for ML (with TrackStatus!):
  Driver  LapNumber                 LapTime  TyreLife Compound  FreshTyre  \
0    VER       15.0  0 days 00:01:58.378000       1.0     SOFT      False   
1    VER       37.0  0 days 00:01:58.490000       1.0     HARD       True   
2    GAS       10.0  0 days 00:02:01.262000       1.0     HARD       True   
3    GAS       26.0  0 days 00:01:59.404000       1.0     HARD       True   
4    GAS       41.0  0 days 00:02:08.544000       1.0     SOFT       True   
5    PER       18.0  0 days 00:01:58.797000       3.0     SOFT      False   
6    PER       35.0  0 days 00:01:58.785000       1.0     HARD       True   
7    ALO       15.0  0 days 00:01:59.692000       1.0     HARD       True   
8    ALO       35.0  0 days 00:01:57.389000       1.0     HARD       True   
9    LEC       14.0  0 days 00:01:58.770000       1.0     HARD       True   

   Position  Stint  TrackStatus            RaceName  SpeedFL  Year  
0       2.0    2.0            1  Bahrain 

In [4]:
# How many rows are complete (no missing values)?
print("📊 Missing Data Analysis:")
print(f"   Total rows: {len(df_features)}")
print(f"   Rows with missing LapTime: {df_features['LapTime'].isnull().sum()}")
print(f"   Rows with missing SpeedFL: {df_features['SpeedFL'].isnull().sum()}")

# How many rows would we lose if we drop missing values?
complete_rows = df_features.dropna()
print(f"\n   Complete rows (no missing): {len(complete_rows)}")
print(f"   Rows we'd lose: {len(df_features) - len(complete_rows)}")
print(f"   Percentage kept: {len(complete_rows)/len(df_features)*100:.1f}%")

# Let's keep the complete rows for now
df_clean = df_features.dropna().copy()

print(f"\n✅ Created clean dataset with {len(df_clean)} rows")

📊 Missing Data Analysis:
   Total rows: 2628
   Rows with missing LapTime: 533
   Rows with missing SpeedFL: 174

   Complete rows (no missing): 2017
   Rows we'd lose: 611
   Percentage kept: 76.8%

✅ Created clean dataset with 2017 rows


In [5]:
# First, check what data type LapTime is
print(f"🔍 LapTime data type: {df_clean['LapTime'].dtype}")
print(f"🔍 Sample LapTime values:")
print(df_clean['LapTime'].head())

# Convert LapTime to timedelta if it's a string
df_clean['LapTime'] = pd.to_timedelta(df_clean['LapTime'])

print(f"\n✅ Converted to timedelta!")
print(f"   New data type: {df_clean['LapTime'].dtype}")

# NOW we can use .dt.total_seconds()
df_clean['LapTime_seconds'] = df_clean['LapTime'].dt.total_seconds()

print(f"\n🔄 Converted to seconds!")
print(f"\n🔍 Sample conversions:")
print(df_clean[['LapTime', 'LapTime_seconds']].head())

print(f"\n📊 LapTime statistics (in seconds):")
print(df_clean['LapTime_seconds'].describe())

# Drop the original LapTime column
df_clean = df_clean.drop('LapTime', axis=1)

print(f"\n✅ LapTime converted and original column dropped!")

🔍 LapTime data type: object
🔍 Sample LapTime values:
0    0 days 00:01:58.378000
1    0 days 00:01:58.490000
2    0 days 00:02:01.262000
3    0 days 00:01:59.404000
4    0 days 00:02:08.544000
Name: LapTime, dtype: object

✅ Converted to timedelta!
   New data type: timedelta64[ns]

🔄 Converted to seconds!

🔍 Sample conversions:
                 LapTime  LapTime_seconds
0 0 days 00:01:58.378000          118.378
1 0 days 00:01:58.490000          118.490
2 0 days 00:02:01.262000          121.262
3 0 days 00:01:59.404000          119.404
4 0 days 00:02:08.544000          128.544

📊 LapTime statistics (in seconds):
count    2017.000000
mean      110.847674
std        15.403476
min        76.832000
25%       101.004000
50%       111.656000
75%       120.528000
max       149.911000
Name: LapTime_seconds, dtype: float64

✅ LapTime converted and original column dropped!


In [6]:
# First, let's see what TrackStatus looks like
print("🚦 TrackStatus sample values:")
print(df_clean['TrackStatus'].head(20))

print(f"\n🚦 Unique TrackStatus values:")
print(df_clean['TrackStatus'].value_counts().head(15))

# Convert TrackStatus to string (in case it's not)
df_clean['TrackStatus'] = df_clean['TrackStatus'].astype(str)

# Create binary features for each status type
df_clean['has_safety_car'] = df_clean['TrackStatus'].str.contains('4', na=False).astype(int)
df_clean['has_vsc'] = df_clean['TrackStatus'].str.contains('6', na=False).astype(int)
df_clean['has_red_flag'] = df_clean['TrackStatus'].str.contains('5', na=False).astype(int)
df_clean['has_yellow'] = df_clean['TrackStatus'].str.contains('2', na=False).astype(int)

print("\n✅ Created track status features!")
print(f"\n📊 Pit stops under different conditions:")
print(f"   Safety Car (4): {df_clean['has_safety_car'].sum()} pit stops ({df_clean['has_safety_car'].sum()/len(df_clean)*100:.1f}%)")
print(f"   VSC (6): {df_clean['has_vsc'].sum()} pit stops ({df_clean['has_vsc'].sum()/len(df_clean)*100:.1f}%)")
print(f"   Red Flag (5): {df_clean['has_red_flag'].sum()} pit stops ({df_clean['has_red_flag'].sum()/len(df_clean)*100:.1f}%)")
print(f"   Yellow Flag (2): {df_clean['has_yellow'].sum()} pit stops ({df_clean['has_yellow'].sum()/len(df_clean)*100:.1f}%)")

# Drop original TrackStatus column
df_clean = df_clean.drop('TrackStatus', axis=1)

print(f"\n✅ Replaced TrackStatus with 4 binary features")
print(f"\n📋 Current columns:")
print(df_clean.columns.tolist())


🚦 TrackStatus sample values:
0       1
1       1
2       1
3       1
4     671
5       1
6       1
7       1
8       1
9       1
10      1
11      1
12      1
13      1
14      1
15    671
16      1
17      1
18    671
19      1
Name: TrackStatus, dtype: int64

🚦 Unique TrackStatus values:
TrackStatus
1      1583
4       205
2        57
41       28
12       26
67       24
671      22
124      12
21       10
7         9
254       9
24        7
64        7
6         6
672       3
Name: count, dtype: int64

✅ Created track status features!

📊 Pit stops under different conditions:
   Safety Car (4): 269 pit stops (13.3%)
   VSC (6): 70 pit stops (3.5%)
   Red Flag (5): 9 pit stops (0.4%)
   Yellow Flag (2): 130 pit stops (6.4%)

✅ Replaced TrackStatus with 4 binary features

📋 Current columns:
['Driver', 'LapNumber', 'TyreLife', 'Compound', 'FreshTyre', 'Position', 'Stint', 'RaceName', 'SpeedFL', 'Year', 'LapTime_seconds', 'has_safety_car', 'has_vsc', 'has_red_flag', 'has_yellow']


In [7]:
# Let's check if we can detect red flag tire changes
# Load the ORIGINAL data again to investigate
df_original = pd.read_csv('data/raw/f1_2023_pit_stops.csv')

# Look for races that had red flags
print("🔍 Checking for red flags in original data...")
print(f"   TrackStatus values containing '5' (red flag):")

red_flag_laps = df_original[df_original['TrackStatus'].astype(str).str.contains('5', na=False)]
print(f"   Total laps with red flag: {len(red_flag_laps)}")

if len(red_flag_laps) > 0:
    print(f"\n🚩 Red flags occurred in these races:")
    print(red_flag_laps[['RaceName', 'LapNumber', 'Driver', 'TrackStatus']].head(20))
    
    # Check if TyreLife resets after red flag
    print(f"\n🔍 Checking TyreLife around red flags:")
    print(red_flag_laps[['Driver', 'LapNumber', 'TyreLife', 'FreshTyre', 'TrackStatus']].head(20))
else:
    print("\n✅ No red flags in our 2023 dataset!")
    print("   This explains why has_red_flag = 0")

🔍 Checking for red flags in original data...
   TrackStatus values containing '5' (red flag):
   Total laps with red flag: 41

🚩 Red flags occurred in these races:
                  RaceName  LapNumber Driver  TrackStatus
95   Australian Grand Prix       55.0    SAR           51
97   Australian Grand Prix        8.0    MAG          451
101  Australian Grand Prix       55.0    DEV           51
104  Australian Grand Prix       55.0    TSU          451
109  Australian Grand Prix       55.0    ZHO          451
124  Australian Grand Prix        8.0    SAI          451
128  Australian Grand Prix        8.0    RUS          451
133  Australian Grand Prix       54.0    BOT           45
134  Australian Grand Prix       55.0    BOT           51
138  Australian Grand Prix       55.0    PIA          451
468       Dutch Grand Prix       64.0    VER          265
479       Dutch Grand Prix       64.0    PER          651
493       Dutch Grand Prix       64.0    STR          514
500       Dutch Grand Pr

### Red Flag Investigation Results

**Question:** Do red flags cause data issues with tire tracking?

**Finding:** ✅ No issues! FastF1 correctly tracks tire changes during red flags.

**Evidence:**
- Australian GP & Dutch GP had red flags in 2023
- TyreLife resets to 1.0 when fresh tires fitted during red flag
- FreshTyre accurately indicates new vs old tires
- Strategic variety: Some drivers changed tires, others didn't

**Conclusion:** Existing features (TyreLife, FreshTyre) already capture red flag tire strategy.
No additional features needed.

In [8]:
print("Compound Types:")
print(df_clean['Compound'].value_counts())

compound_mapping = {
    'SOFT': 1,
    'MEDIUM': 2,
    'HARD': 3,
    'INTERMEDIATE': 4,
    'WET': 5
}

df_clean['Compound_encoded'] = df_clean['Compound'].map(compound_mapping)

print("\n✅ Encoded Compound types:")
print ("encoding mapping:")
for compound, code in sorted(compound_mapping.items(), key=lambda x: x[1]):
    count = (df_clean['Compound'] == compound).sum()
    pct = count / len(df_clean) * 100
    print(f"  {code} = {compound:15} ->  {count:3} pit stops ({pct:5.1f}%)")

print ("verification sample:")
print(df_clean[['Compound', 'Compound_encoded']].head(10))

unmapped = df_clean['Compound_encoded'].isnull().sum()

if unmapped > 0:
    print(f"\n⚠️ Warning: {unmapped} compounds not mapped!")
    unmapped_compounds = df_clean[df_clean['Compound_encoded'].isnull()]['Compound'].tolist()
    print(f"   Unmapped values: {unmapped_compounds}")
else:
    print("\n✅ All Compound values successfully mapped!")

# Drop original Compound column
df_clean = df_clean.drop('Compound', axis=1)

print(f"\n✅ Dropped original Compound column")
print(f"\n📋 Final columns: {len(df_clean.columns)}")

Compound Types:
Compound
HARD            989
MEDIUM          612
SOFT            281
INTERMEDIATE    130
WET               5
Name: count, dtype: int64

✅ Encoded Compound types:
encoding mapping:
  1 = SOFT            ->  281 pit stops ( 13.9%)
  2 = MEDIUM          ->  612 pit stops ( 30.3%)
  3 = HARD            ->  989 pit stops ( 49.0%)
  4 = INTERMEDIATE    ->  130 pit stops (  6.4%)
  5 = WET             ->    5 pit stops (  0.2%)
verification sample:
  Compound  Compound_encoded
0     SOFT                 1
1     HARD                 3
2     HARD                 3
3     HARD                 3
4     SOFT                 1
5     SOFT                 1
6     HARD                 3
7     HARD                 3
8     HARD                 3
9     HARD                 3

✅ All Compound values successfully mapped!

✅ Dropped original Compound column

📋 Final columns: 15


In [9]:
from sklearn.preprocessing import LabelEncoder

print("Remaining categorical columns to encode:")
print(f"Driver: {df_clean['Driver'].nunique()} unique values")
print(f"RaceName: {df_clean['RaceName'].nunique()} unique values")

driver_encoder = LabelEncoder()
race_encoder = LabelEncoder()

df_clean['Driver_encoded'] = driver_encoder.fit_transform(df_clean['Driver'])
df_clean['RaceName_encoded'] = race_encoder.fit_transform(df_clean['RaceName'])

print("\n driver encoding sample:")
print(df_clean[['Driver', 'Driver_encoded']].head(10))

print("\n race encoding sample:")
print(df_clean[['RaceName', 'RaceName_encoded']].head(10))

df_clean = df_clean.drop(['Driver', 'RaceName'], axis=1)

print("\n Dropped original Driver and RaceName columns")
print(f"\n📋 Final dataset columns: {len(df_clean.columns)}")
print(df_clean.columns.tolist())

Remaining categorical columns to encode:
Driver: 33 unique values
RaceName: 33 unique values

 driver encoding sample:
  Driver  Driver_encoded
0    VER              30
1    VER              30
2    GAS               6
3    GAS               6
4    GAS               6
5    PER              21
6    PER              21
7    ALO               2
8    ALO               2
9    LEC              15

 race encoding sample:
             RaceName  RaceName_encoded
0  Bahrain Grand Prix                 5
1  Bahrain Grand Prix                 5
2  Bahrain Grand Prix                 5
3  Bahrain Grand Prix                 5
4  Bahrain Grand Prix                 5
5  Bahrain Grand Prix                 5
6  Bahrain Grand Prix                 5
7  Bahrain Grand Prix                 5
8  Bahrain Grand Prix                 5
9  Bahrain Grand Prix                 5

 Dropped original Driver and RaceName columns

📋 Final dataset columns: 15
['LapNumber', 'TyreLife', 'FreshTyre', 'Position', 'Stint', 'Speed

In [10]:
# Year doesn't need encoding - it's already numeric!
print("📅 Checking Year column...")

print(f"\n🔍 Year statistics:")
print(f"   Data type: {df_clean['Year'].dtype}")
print(f"   Unique values: {sorted(df_clean['Year'].unique())}")
print(f"   Distribution:")
print(df_clean['Year'].value_counts().sort_index())

# Year is already numeric (2020, 2021, 2023, 2024)
# Perfect for ML - no encoding needed! ✅

print(f"\n✅ Year ready for ML (already numeric)")

📅 Checking Year column...

🔍 Year statistics:
   Data type: int64
   Unique values: [2020, 2021, 2023, 2024]
   Distribution:
Year
2020    472
2021    599
2023    725
2024    221
Name: count, dtype: int64

✅ Year ready for ML (already numeric)


In [11]:
# Check if has_red_flag is all zeros
print("🔍 Checking has_red_flag values:")
print(df_clean['has_red_flag'].value_counts())

# Since it's all zeros (no variance), remove it
if 'has_red_flag' in df_clean.columns:
    if df_clean['has_red_flag'].sum() == 0:
        print("\n❌ Removing has_red_flag (all zeros, no predictive value)")
        df_clean = df_clean.drop('has_red_flag', axis=1)
    else:
        print(f"\n✅ Keeping has_red_flag ({df_clean['has_red_flag'].sum()} pit stops under red flag)")

print(f"\n📊 Final feature count: {len(df_clean.columns)}")
print(f"📋 Final columns:")
for i, col in enumerate(df_clean.columns, 1):
    print(f"   {i:2}. {col}")

# Check data types (should all be numeric now!)
print(f"\n🔍 Data types:")
print(df_clean.dtypes)

# Check for any remaining missing values
missing = df_clean.isnull().sum()
if missing.sum() == 0:
    print("\n✅ No missing values!")
else:
    print(f"\n⚠️ Missing values found:")
    print(missing[missing > 0])

print(f"\n🎉 Feature engineering complete!")
print(f"   Rows: {len(df_clean)}")
print(f"   Features: {len(df_clean.columns)}")

🔍 Checking has_red_flag values:
has_red_flag
0    2008
1       9
Name: count, dtype: int64

✅ Keeping has_red_flag (9 pit stops under red flag)

📊 Final feature count: 15
📋 Final columns:
    1. LapNumber
    2. TyreLife
    3. FreshTyre
    4. Position
    5. Stint
    6. SpeedFL
    7. Year
    8. LapTime_seconds
    9. has_safety_car
   10. has_vsc
   11. has_red_flag
   12. has_yellow
   13. Compound_encoded
   14. Driver_encoded
   15. RaceName_encoded

🔍 Data types:
LapNumber           float64
TyreLife            float64
FreshTyre              bool
Position            float64
Stint               float64
SpeedFL             float64
Year                  int64
LapTime_seconds     float64
has_safety_car        int32
has_vsc               int32
has_red_flag          int32
has_yellow            int32
Compound_encoded      int64
Driver_encoded        int32
RaceName_encoded      int32
dtype: object

✅ No missing values!

🎉 Feature engineering complete!
   Rows: 2017
   Features: 15


In [12]:
# Save the processed, ML-ready dataset
output_file = 'data/processed/ml_features_update.csv'
df_clean.to_csv(output_file, index=False)

print(f"✅ Saved processed data!")
print(f"   File: {output_file}")
print(f"   Rows: {len(df_clean)}")
print(f"   Columns: {len(df_clean.columns)}")

# Show summary statistics
print(f"\n📊 Dataset summary:")
print(df_clean.describe())

print(f"\n🎯 Ready for model training!")

✅ Saved processed data!
   File: data/processed/ml_features_update.csv
   Rows: 2017
   Columns: 15

📊 Dataset summary:
         LapNumber     TyreLife     Position        Stint      SpeedFL  \
count  2017.000000  2017.000000  2017.000000  2017.000000  2017.000000   
mean     27.610312     1.921666    12.439266     2.728805   257.236986   
std      15.493722     2.902143     5.571452     0.911962    40.487975   
min       1.000000     1.000000     1.000000     1.000000    47.000000   
25%      15.000000     1.000000     8.000000     2.000000   244.000000   
50%      27.000000     1.000000    14.000000     3.000000   271.000000   
75%      38.000000     1.000000    17.000000     3.000000   283.000000   
max      79.000000    34.000000    20.000000     8.000000   337.000000   

              Year  LapTime_seconds  has_safety_car      has_vsc  \
count  2017.000000      2017.000000     2017.000000  2017.000000   
mean   2021.813585       110.847674        0.133366     0.034705   
std      